In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torchvision
import torchvision.datasets as datasets
import sys
import numpy as np
import torch.utils.data as utils
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
sys.path.append("../../Dataset-REPAIR")
from utils.datasets import ColoredDataset
import pickle as pkl

from os.path import join as oj
import matplotlib.pyplot as plt
%matplotlib inline
import os

In [3]:
import torch
import torchvision
import torchvision.datasets as datasets
import sys
import matplotlib.pyplot as plt
import numpy as np
import torch.utils.data as utils
from sklearn.metrics import auc,average_precision_score, roc_curve,roc_auc_score

In [4]:
from torch import nn

In [18]:
device = torch.device("cuda")

In [19]:
import torchvision.models as models

model = models.vgg16(pretrained=True).to(device)
model.classifier[-1] = nn.Linear(4096, 2)
model.load_state_dict(torch.load('cancer_prim.pt'))

In [27]:
data_root  = "../../../datasets/ISIC"
data_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])
img_dataset = datasets.ImageFolder(data_root,transform=data_transform)
num_total = len(img_dataset)
num_train = int(0.8 * num_total)
num_val = int(0.1 * num_total)
num_test = num_total - num_train - num_val
torch.manual_seed(0);
train_dataset, test_dataset, val_dataset= torch.utils.data.random_split(img_dataset, [num_train, num_test, num_val])

In [28]:
datasets = {'train' : train_dataset, 'test':test_dataset, 'val': val_dataset}
dataloaders = {x: torch.utils.data.DataLoader(datasets[x], batch_size=64,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'test','val']}

In [ ]:

not_cancer_ratio = np.asarray(train_dataset.dataset.targets).mean() 
cancer_ratio = 1- not_cancer_ratio
cancer_weight = 1/cancer_ratio
not_cancer_weight = 1/ not_cancer_ratio
weights = np.asarray([cancer_weight, not_cancer_weight])
weights /= weights.sum()
weights = torch.tensor(weights).to(device)

In [56]:
model.eval()
y = []
y_hat = []
softmax= torch.nn.Softmax()
for inputs, labels in dataloaders['val']:
    y_hat.append((1-labels).cpu().numpy())
    y.append(torch.nn.Softmax()( model(inputs.cuda()))[:,0].detach().cpu().numpy())
y_hat = np.concatenate( y_hat, axis=0 )
y = np.concatenate( y, axis=0 )

/home/lauri/.conda/envs/gpu_usage/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  import sys


In [70]:
train_dataset.dataset.classes

['cancer', 'not_cancer']

In [97]:
roc_auc_score(y_hat, y)

0.9242712774497387

In [103]:
num_fp = (y > 0.5)[np.where(y_hat == 0)].sum()
num_tp = (y > 0.5)[np.where(y_hat == 1)].sum()

num_fn = (y < 0.5)[np.where(y_hat == 1)].sum()
num_tn = (y < 0.5)[np.where(y_hat == 0)].sum()


In [122]:
print("Precision: {:.2f}".format(num_tp/(num_tp+num_fp)))
print("Recall: {:.2f}".format(num_tp/(num_tp+num_fn)))

Precision: 0.45
Recall: 0.70


In [124]:
print("For not cancer:")
print("Precision: {:.2f}".format(num_tn/(num_tn+num_fn)))
print("Recall: {:.2f}".format(num_tn/(num_tn+num_fp)))

For not cancer:
Precision: 0.96
Recall: 0.90
